## Deriving the Black-Scholes-Merton Equation
Based on the very informative YouTube video, [FN452 Deriving the Black-Scholes-Merton Equation](https://www.youtube.com/watch?v=IynFtIQ6HaI) by Nattakit Chokwattananuwat. 